COLOR, TEXTURE AND SHAPE SIMILARITY

In [ ]:
!pip install opencv-python-headless numpy scikit-image

In [ ]:
# Import libraries and define utility functions

!pip install opencv-python-headless numpy scikit-image

import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from scipy.spatial.distance import euclidean
from google.colab import files
import os
import pandas as pd
import tarfile
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as T
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt


def upload_images():
    uploaded = files.upload()
    return list(uploaded.keys())

def read_image(file_path):
    return cv2.imread(file_path)

# Define similarity functions

def calculate_color_similarity(image1, image2):
    def calculate_color_histogram(image):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
        cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
        return hist.flatten()

    hist1 = calculate_color_histogram(image1)
    hist2 = calculate_color_histogram(image2)
    bhattacharyya_dist = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)

    return bhattacharyya_dist

def calculate_shape_similarity(image1, image2):
    def hu_moments(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        moments = cv2.moments(gray)
        hu_moments = cv2.HuMoments(moments).flatten()
        return -np.sign(hu_moments) * np.log10(np.abs(hu_moments))

    hu1 = hu_moments(image1)
    hu2 = hu_moments(image2)
    hu_dist = euclidean(hu1, hu2)

    return hu_dist

def calculate_texture_similarities(image1, image2):
    # LBP comparison using KL divergence
    def lbp_histogram(image, num_points=24, radius=8, eps=1e-7):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, num_points, radius, method="uniform")
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

    def kl_divergence(p, q):
        return np.sum(np.where(p != 0, p * np.log(p / q), 0))

    lbp1 = lbp_histogram(image1)
    lbp2 = lbp_histogram(image2)
    lbp_dist = kl_divergence(lbp1, lbp2)

    # Gabor feature comparison
    def gabor_features(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = []
        for theta in range(4):
            theta = theta / 4. * np.pi
            for sigma in (1, 3):
                for frequency in (0.05, 0.25):
                    filt_real, filt_imag = gabor(gray, frequency=frequency, theta=theta, sigma_x=sigma, sigma_y=sigma)
                    features.append(filt_real.mean())
                    features.append(filt_imag.mean())
        return np.array(features)

    gabor1 = gabor_features(image1)
    gabor2 = gabor_features(image2)
    gabor_dist = euclidean(gabor1, gabor2)

    return lbp_dist, gabor_dist

# Main function to compare images

def compare_images():
    print("Please upload the images you want to compare:")
    image_files = upload_images()

    if len(image_files) != 2:
        print("Please upload exactly two images.")
        return

    image1 = read_image(image_files[0])
    image2 = read_image(image_files[1])

    if image1 is None or image2 is None:
        print("Error reading one or both images.")
        return

    color_dist = calculate_color_similarity(image1, image2)
    shape_dist = calculate_shape_similarity(image1, image2)
    lbp_dist, gabor_dist = calculate_texture_similarities(image1, image2)

    print(f"Comparing {image_files[0]} and {image_files[1]}:")
    print(f"Color Similarity (Bhattacharyya Distance): {color_dist:.4f}")
    print(f"Shape Similarity (Hu Moment, Euclidean Distance): {shape_dist:.4f}")
    print(f"Texture Similarity (LBP, KL Divergence): {lbp_dist:.4f}")
    print(f"Texture Similarity (Gabor, Euclidean Distance): {gabor_dist:.4f}")

# Run this function to compare images
compare_images()

!pip install opencv-python-headless numpy scikit-image annoy

import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from scipy.spatial.distance import euclidean
from google.colab import files
from annoy import AnnoyIndex
import random

import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from annoy import AnnoyIndex
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to the earrings folder
base_path = '/content/drive/MyDrive/earrings/'

def read_image(file_path):
    return cv2.imread(file_path)

def calculate_color_histogram(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
    cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
    return hist.flatten()

def calculate_hu_moments(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).flatten()
    return -np.sign(hu_moments) * np.log10(np.abs(hu_moments))

def calculate_lbp_histogram(image, num_points=24, radius=8, eps=1e-7):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, num_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    return hist

def calculate_gabor_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for frequency in (0.05, 0.25):
                filt_real, filt_imag = gabor(gray, frequency=frequency, theta=theta, sigma_x=sigma, sigma_y=sigma)
                features.append(filt_real.mean())
                features.append(filt_imag.mean())
    return np.array(features)

def extract_features(image):
    color_hist = calculate_color_histogram(image)
    hu_moments = calculate_hu_moments(image)
    lbp_hist = calculate_lbp_histogram(image)
    gabor_features = calculate_gabor_features(image)
    return np.concatenate([color_hist, hu_moments, lbp_hist, gabor_features])

def build_annoy_index(features, n_trees=10, index_file='image_similarity_index.ann'):
    feature_length = features.shape[1]
    index = AnnoyIndex(feature_length, 'angular')
    for i, feature in enumerate(features):
        index.add_item(i, feature)
    index.build(n_trees)
    index.save(index_file)
    return index

def find_similar_images(index, features, query_index, n=5):
    similar_indices = index.get_nns_by_vector(features[query_index], n+1)[1:]  # exclude self
    return [(i, index.get_distance(query_index, i)) for i in similar_indices]

def process_images_with_annoy():
    image_files = [f for f in os.listdir(base_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    features = []
    for img_file in image_files:
        img_path = os.path.join(base_path, img_file)
        image = read_image(img_path)
        if image is not None:
            feature = extract_features(image)
            features.append(feature)
        else:
            print(f"Failed to read image: {img_file}")

    features = np.array(features)

    # Build Annoy index
    index = build_annoy_index(features)

    # Example: Find similar images for the first image
    query_index = 0
    similar_images = find_similar_images(index, features, query_index)

    # Print results
    print(f"Images similar to {image_files[query_index]}:")
    for idx, distance in similar_images:
        print(f"Similar image: {image_files[idx]}, Distance: {distance}")

# Run the process
process_images_with_annoy()




###ANNOY INDEX

In [ ]:
!pip install opencv-python-headless numpy scikit-image annoy

import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from scipy.spatial.distance import euclidean
from google.colab import files
from annoy import AnnoyIndex
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=550734 sha256=cb3466d17f0387ad24eb8b2063a809b76aeaa62596cc21ff2ac02829366ec435
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from annoy import AnnoyIndex
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to the earrings folder
base_path = '/content/drive/MyDrive/earrings/'

def read_image(file_path):
    return cv2.imread(file_path)

def calculate_color_histogram(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
    cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
    return hist.flatten()

def calculate_hu_moments(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).flatten()
    return -np.sign(hu_moments) * np.log10(np.abs(hu_moments))

def calculate_lbp_histogram(image, num_points=24, radius=8, eps=1e-7):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, num_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    return hist

def calculate_gabor_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for frequency in (0.05, 0.25):
                filt_real, filt_imag = gabor(gray, frequency=frequency, theta=theta, sigma_x=sigma, sigma_y=sigma)
                features.append(filt_real.mean())
                features.append(filt_imag.mean())
    return np.array(features)

def extract_features(image):
    color_hist = calculate_color_histogram(image)
    hu_moments = calculate_hu_moments(image)
    lbp_hist = calculate_lbp_histogram(image)
    gabor_features = calculate_gabor_features(image)
    return np.concatenate([color_hist, hu_moments, lbp_hist, gabor_features])

def build_annoy_index(features, n_trees=10, index_file='image_similarity_index.ann'):
    feature_length = features.shape[1]
    index = AnnoyIndex(feature_length, 'angular')
    for i, feature in enumerate(features):
        index.add_item(i, feature)
    index.build(n_trees)
    index.save(index_file)
    return index

def find_similar_images(index, features, query_index, n=5):
    similar_indices = index.get_nns_by_vector(features[query_index], n+1)[1:]  # exclude self
    return [(i, index.get_distance(query_index, i)) for i in similar_indices]

def process_images_with_annoy():
    image_files = [f for f in os.listdir(base_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    features = []
    for img_file in image_files:
        img_path = os.path.join(base_path, img_file)
        image = read_image(img_path)
        if image is not None:
            feature = extract_features(image)
            features.append(feature)
        else:
            print(f"Failed to read image: {img_file}")

    features = np.array(features)

    # Build Annoy index
    index = build_annoy_index(features)

    # Example: Find similar images for the first image
    query_index = 0
    similar_images = find_similar_images(index, features, query_index)

    # Print results
    print(f"Images similar to {image_files[query_index]}:")
    for idx, distance in similar_images:
        print(f"Similar image: {image_files[idx]}, Distance: {distance}")

# Run the process
process_images_with_annoy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Images similar to 502415SAEAGA09_1 (2).jpg:
Similar image: 502415SAEAGA09_1 (1).jpg, Distance: 0.0
Similar image: 502415SAEAGA09_1.jpg, Distance: 0.0
Similar image: 511183DQJABA18_1.jpg, Distance: 0.007114844862371683
Similar image: 500487SAAABA15_1.jpg, Distance: 0.008710759691894054
Similar image: 511417DDPABA00_1.jpg, Distance: 0.009248176589608192


###Ball Tree

In [ ]:
!pip install scikit-learn

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from sklearn.neighbors import BallTree
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to the earrings folder in your drive
base_path = '/content/drive/MyDrive/earrings/'

def read_image(file_path):
    return cv2.imread(file_path)

def calculate_color_histogram(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
    cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
    return hist.flatten()

def calculate_hu_moments(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).flatten()
    return -np.sign(hu_moments) * np.log10(np.abs(hu_moments))

def calculate_lbp_histogram(image, num_points=24, radius=8, eps=1e-7):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, num_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    return hist

def calculate_gabor_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for frequency in (0.05, 0.25):
                filt_real, filt_imag = gabor(gray, frequency=frequency, theta=theta, sigma_x=sigma, sigma_y=sigma)
                features.append(filt_real.mean())
                features.append(filt_imag.mean())
    return np.array(features)

def extract_features(image):
    color_hist = calculate_color_histogram(image)
    hu_moments = calculate_hu_moments(image)
    lbp_hist = calculate_lbp_histogram(image)
    gabor_features = calculate_gabor_features(image)
    return np.concatenate([color_hist, hu_moments, lbp_hist, gabor_features])

def build_balltree(features):
    return BallTree(features)

def find_similar_images(tree, features, query_index, n=5):
    distances, indices = tree.query(features[query_index:query_index+1], k=n+1)
    return list(zip(indices[0][1:], distances[0][1:]))  # exclude self

def process_images_with_balltree():
    print(f"Searching for images in: {base_path}")
    image_files = [f for f in os.listdir(base_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(f"No image files found in {base_path}")
        return

    print(f"Found {len(image_files)} images.")

    features = []
    for img_file in image_files:
        img_path = os.path.join(base_path, img_file)
        image = read_image(img_path)
        if image is not None:
            feature = extract_features(image)
            features.append(feature)
        else:
            print(f"Failed to read image: {img_file}")

    features = np.array(features)

    # Build BallTree
    tree = build_balltree(features)

    # Example: Find similar images for the first image
    query_index = 0
    similar_images = find_similar_images(tree, features, query_index)

    # Print results
    print(f"Images similar to {image_files[query_index]}:")
    for idx, distance in similar_images:
        print(f"Similar image: {image_files[idx]}, Distance: {distance}")

# Run the process
process_images_with_balltree()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Searching for images in: /content/drive/MyDrive/earrings/
Found 396 images.
Images similar to 502415SAEAGA09_1 (2).jpg:
Similar image: 502415SAEAGA09_1 (1).jpg, Distance: 0.0
Similar image: 502415SAEAGA09_1 (2).jpg, Distance: 0.0
Similar image: 502014SFDAGA52_1.jpg, Distance: 5.866915218908919
Similar image: 501055SHCABB09_1.jpg, Distance: 6.255253685312517
Similar image: 511417DDPABA00_1.jpg, Distance: 6.377874593903574


Barlow Twins Embeddings

In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_tfms = T.Compose([
    T.Resize(224),
    T.ToTensor(),
    T.Normalize(*imagenet_stats,inplace=True),
])

valid_tfms = T.Compose([
    T.Resize(224),
    T.ToTensor(),
    T.Normalize(*imagenet_stats)
])



In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Set the path to the earrings folder in your drive
data_dir = '/content/drive/MyDrive/earrings/'

dataset = ImageFolder(data_dir, transform=train_tfms)

random_seed = 42
torch.manual_seed(random_seed);

val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size

# Split into train and validation dataset
train_ds, val_ds = random_split(dataset, [train_size, val_size])

batch_size=4

# Training and validation dataloaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=1, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, num_workers=1, pin_memory=True)

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

def shown_example(img, label):
    '''display an image with class name and label'''
    print('Label:', train_ds.dataset.classes[label], '('+str(label)+')')
    #check for the number of channels
    if (img.size()[0] == 3):
        unorm_image = unorm(img)
        plt.imshow(unorm_image.permute(1,2,0))
    else:
        unorm_image = unorm(img)
        plt.imshow(unorm_image.squeeze())

def show_batch(dl):
    '''display a batch of images'''
    for images, labels in dl:
        norm_images = images.clone()
        unorm_images = []
        for img in images:
            unorm_image = unorm(img)
            unorm_images.append(unorm_image)
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(24,18))
        axs[0].set_xticks([])
        axs[0].set_yticks([])
        axs[1].set_xticks([])
        axs[1].set_yticks([])
        axs[0].imshow(make_grid(norm_images, nrow=8).permute(1,2,0))
        axs[1].imshow(make_grid(unorm_images, nrow=8).permute(1,2,0))
        break

unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [ ]:
shown_example(*train_ds[0])

In [ ]:
model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
model.cuda();

In [ ]:


from torchvision.io.image import read_image
from PIL import Image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.cams import ScoreCAM, SSCAM, ISCAM
from torchcam.utils import overlay_mask

def plot_cam(img_path, cam_extractor, model):
    # Get your input
    img = read_image(img_path)

    # Preprocess it for your chosen model
    input_tensor = normalize(resize(img, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    # Preprocess your data and feed it to the model
    out = model(input_tensor.unsqueeze(0).cuda())
    # Retrieve the CAM by passing the class index and the model output
    activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

    # Visualize the raw CAM
    axs[0].imshow(activation_map.cpu().numpy());plt.axis('off'); axs[0].set_xticks([]); axs[0].set_yticks([])

    # Resize the CAM and overlay it
    result = overlay_mask(to_pil_image(img), to_pil_image(activation_map, mode='F'), alpha=0.5)
    # Display it
    axs[1].imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()

In [ ]:
cam_extractor = ScoreCAM(model)

In [ ]:
@torch.no_grad()
def predict_dl(dl, model):
    torch.cuda.empty_cache()
    batch_preds = []
    for xb, _ in tqdm(dl):
        # Get predictions from model
        yb = model(xb.cuda())
        # Pick index with highest probability
        _, preds  = torch.max(yb, dim=1)
        batch_preds.append(preds)
    batch_preds = torch.cat(batch_preds)
    return batch_preds

In [ ]:
class SaveFeatures():
    features=None
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_fn)
        self.features = None
    def hook_fn(self, module, input, output):
        output = output.squeeze(3)
        output = output.squeeze(2)
        out = output.detach().cpu().numpy()
        if isinstance(self.features, type(None)):
            self.features = out
        else:
            self.features = np.row_stack((self.features, out))
    def remove(self):
        self.hook.remove()

In [ ]:
sf = SaveFeatures(model.avgpool)

In [ ]:
dl = DataLoader(dataset, batch_size, shuffle=False, num_workers=4, pin_memory=True)
predictions = predict_dl(dl, model)

In [ ]:
len(predictions)

In [ ]:
img_path = [x[0] for x in (list(dl.dataset.imgs))]
label = [dl.dataset.classes[x[1]] for x in (list(dl.dataset.imgs))]
label_id = [x[1] for x in (list(dl.dataset.imgs))]

In [ ]:
len(img_path), len(label), len(label_id)

In [ ]:
df_new = pd.DataFrame({'img_path': img_path, 'label': label, 'label_id': label_id})
df_new.head()

In [ ]:
df_new['label'].value_counts()

In [ ]:
array = sf.features
x=array.tolist()
df_new['img_repr'] = x

In [ ]:
len(x[0])

In [ ]:
df_new.to_csv('barlow_twins_earrings_embeddings.csv')